In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 44.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.1 MB/s eta 0:00:00:00:01


In [5]:
import os
import json
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
# Set up SageMaker session and role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [7]:
# Specify your S3 bucket and prefix for training data
bucket = 'sagemaker-studio-734555379704-26qtsi8abn9'
prefix = ''

In [9]:
# Load your training data from a JSON file
data_file = 'dummy_data.json'
with open(data_file, 'r') as f:
    train_data = json.load(f)

In [10]:
# Preprocess data
tokenized_descriptions = []
for data_point in train_data:
    description = data_point["description"]
    keywords = " ".join(data_point["keywords"])
    tokenized_description = f"{keywords} {description}"
    tokenized_descriptions.append(tokenized_description)

In [11]:
# Save tokenized descriptions to a file
train_file = 'train_data.txt'
with open(train_file, 'w') as f:
    for item in tokenized_descriptions:
        f.write("%s\n" % item)

In [ ]:
# Upload data to S3
train_channel = sagemaker_session.upload_data(train_file, bucket=bucket, key_prefix=prefix + '/train')

# Load mistralai/Mistral-7B-v0.1 tokenizer and model
model_name = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

In [ ]:
# Save the tokenizer and model to a directory
model_dir = 'model'
os.makedirs(model_dir, exist_ok=True)
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

# Upload model and tokenizer to S3
model_channel = sagemaker_session.upload_data(model_dir, bucket=bucket, key_prefix=prefix + '/model')

# Define hyperparameters for training
hyperparameters = {
    "epochs": 5,
    "train_file": "train_data.txt",
    "model_name": model_name
}

# Specify the SageMaker PyTorch estimator
estimator = PyTorch(entry_point='train_lm.py',
                    source_dir='source',
                    role=role,
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.p3.2xlarge',
                    hyperparameters=hyperparameters)

# Train the model
estimator.fit({'train': train_channel, 'model': model_channel})
